# ETL: Fase de preparacion, procesamiento y limpieza de los datos

### En esta fase estaremos explorando el archivo en bruto para ver que tipo de informacion tenemos en cada hoja y columna. Identificar variables claves que nos ayuden a responder preguntas definidas

### Determinar la estructura de los datos, observar si hay datos faltantes o valores anomalos, y corregirlos de ser necesarios.
---

In [1]:
# Iniciamos importando las librerias necesarias.
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Guardamos en una variable la ruta de nuestro dataset.
file_path = './Datasets/Internet.xlsx'

In [3]:
# Cargamos el archivo e imprimimos los nombres de las hojas para entender su estructura.
archivo_excel = pd.ExcelFile(file_path)
hojas = archivo_excel.sheet_names
hojas

['Acc_vel_loc_sinrangos',
 'Velocidad_sin_Rangos',
 'Accesos_tecnologia_localidad',
 'Velocidad % por prov',
 'Totales VMD',
 'Totales Accesos Por Tecnología',
 'Accesos Por Tecnología',
 'Penetración-poblacion',
 'Penetracion-hogares',
 'Penetracion-totales',
 'Totales Accesos por rango',
 'Accesos por rangos',
 'Dial-BAf',
 'Totales Dial-BAf',
 'Ingresos ']

In [4]:
# Crear un diccionario para almacenar cada hoja como un DataFrame individual
dataframes = {hoja: archivo_excel.parse(hoja) for hoja in hojas}
# Imprimir cada dataframe
for dataframe in dataframes.keys():
    print(dataframe)


Acc_vel_loc_sinrangos
Velocidad_sin_Rangos
Accesos_tecnologia_localidad
Velocidad % por prov
Totales VMD
Totales Accesos Por Tecnología
Accesos Por Tecnología
Penetración-poblacion
Penetracion-hogares
Penetracion-totales
Totales Accesos por rango
Accesos por rangos
Dial-BAf
Totales Dial-BAf
Ingresos 


#### Procederemos a observar si hay valores faltantes y su debido tratado para cada dataframe

In [5]:
# Imprimimos los valores iniciales de nuestro 1er dataframe
dataframes['Acc_vel_loc_sinrangos'].head()

,Partido,Localidad,link Indec,Velocidad (Mbps),Provincia,Accesos
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.00,1.0
1,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.50,2.0
2,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.75,19.0
3,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,3.00,85.0
4,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,3.50,145.0


In [6]:
# Se procede a renombrar las columnas correctamente
dataframes['Acc_vel_loc_sinrangos'].rename(columns= {'Partido': 'Provincia', 'Localidad': 'Partido', 'link Indec': 'Localidad', 'Velocidad (Mbps)': 'Link Indec', 'Provincia': 'Velocidad (Mbps)'}, inplace=True)
dataframes['Acc_vel_loc_sinrangos'].head()

,Provincia,Partido,Localidad,Link Indec,Velocidad (Mbps),Accesos
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.00,1.0
1,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.50,2.0
2,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.75,19.0
3,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,3.00,85.0
4,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,3.50,145.0


In [7]:
primer_dataframe = list(dataframes.keys())[0]
print('1er dataframe: ', primer_dataframe)
print('-' * 50)
print('Cantidad de nulos por columna: \n', dataframes['Acc_vel_loc_sinrangos'].isnull().sum())
print('-' * 50)
print('Cantidad de filas duplicadas: ', dataframes['Acc_vel_loc_sinrangos'].duplicated().sum())
print('-' * 50)
print('Tipos de datos: \n', dataframes['Acc_vel_loc_sinrangos'].dtypes)

1er dataframe:  Acc_vel_loc_sinrangos
--------------------------------------------------
Cantidad de nulos por columna: 
 Provincia           0
Partido             0
Localidad           1
Link Indec          0
Velocidad (Mbps)    0
Accesos             7
dtype: int64
--------------------------------------------------
Cantidad de filas duplicadas:  0
--------------------------------------------------
Tipos de datos: 
 Provincia            object
Partido              object
Localidad            object
Link Indec           object
Velocidad (Mbps)    float64
Accesos             float64
dtype: object


In [8]:
# Se procedio a identificar cuales eran las filas con valores nulos
fila, columna = np.where(dataframes['Acc_vel_loc_sinrangos'].isna())
dataframes['Acc_vel_loc_sinrangos'].iloc[fila]

,Provincia,Partido,Localidad,Link Indec,Velocidad (Mbps),Accesos
7172,CABA,Ciudad Autónoma de Buenos Aires,NaN,Sin Datos,300.0,-5582.0
10510,CORRIENTES,Ituzaingó,Colonia Liebig's,18084010,0.0,NaN
11288,ENTRE RIOS,Tala,Rosario del Tala,30091100,0.0,NaN
14784,SALTA,Cafayate,Tolombón,66021020,0.0,NaN
15288,SALTA,San Carlos,Animaná,66154020,0.0,NaN
15294,SALTA,San Carlos,San Carlos,66154040,0.0,NaN
17235,SANTA FE,Rosario,General Lagos,82084150,0.0,NaN
18090,SANTIAGO DEL ESTERO,Choya,Frías,86063040,0.0,NaN


Agregar la localidad y el link. Arreglar el acceso

In [9]:
dataframes['Acc_vel_loc_sinrangos']['Provincia'].unique()

array(['BUENOS AIRES', 'CABA', 'CATAMARCA', 'CHACO', 'CHUBUT', 'CORDOBA',
       'CORRIENTES', 'ENTRE RIOS', 'FORMOSA', 'JUJUY', 'LA PAMPA',
       'LA RIOJA', 'MENDOZA', 'MISIONES', 'NEUQUEN', 'RIO NEGRO', 'SALTA',
       'SAN JUAN', 'SAN LUIS', 'SANTA CRUZ', 'SANTA FE',
       'SANTIAGO DEL ESTERO', 'TIERRA DEL FUEGO', 'TUCUMAN'], dtype=object)

In [10]:
dataframes['Acc_vel_loc_sinrangos']['Provincia'] = dataframes['Acc_vel_loc_sinrangos']['Provincia'].str.title()
dataframes['Acc_vel_loc_sinrangos']['Provincia'].unique()

array(['Buenos Aires', 'Caba', 'Catamarca', 'Chaco', 'Chubut', 'Cordoba',
       'Corrientes', 'Entre Rios', 'Formosa', 'Jujuy', 'La Pampa',
       'La Rioja', 'Mendoza', 'Misiones', 'Neuquen', 'Rio Negro', 'Salta',
       'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe',
       'Santiago Del Estero', 'Tierra Del Fuego', 'Tucuman'], dtype=object)

### ETL para KPI 1 (Penetracion-hogares, Accesos_tecnologia_localicad, Accesos por tecnologia)

In [11]:
dataframes['Penetracion-hogares'].head()

,Año,Trimestre,Provincia,Accesos por cada 100 hogares
0,2024,2,Buenos Aires,79.84
1,2024,2,Capital Federal,116.37
2,2024,2,Catamarca,68.81
3,2024,2,Chaco,44.06
4,2024,2,Chubut,86.33


In [12]:
# Se procede a revisar los valores nulos por columna, filas duplicadas y tipo de dato
noveno_dataframe = list(dataframes.keys())[8]
print('9no dataframe: ', noveno_dataframe)
print('-' * 50)
print('Cantidad de nulos por columna: \n', dataframes['Penetracion-hogares'].isnull().sum())
print('-' * 50)
print('Cantidad de filas duplicadas: ', dataframes['Penetracion-hogares'].duplicated().sum())
print('-' * 50)
print('Tipos de datos: \n', dataframes['Penetracion-hogares'].dtypes)

9no dataframe:  Penetracion-hogares
--------------------------------------------------
Cantidad de nulos por columna: 
 Año                             0
Trimestre                       0
Provincia                       0
Accesos por cada 100 hogares    0
dtype: int64
--------------------------------------------------
Cantidad de filas duplicadas:  0
--------------------------------------------------
Tipos de datos: 
 Año                               int64
Trimestre                         int64
Provincia                        object
Accesos por cada 100 hogares    float64
dtype: object


In [13]:
dataframes['Penetracion-hogares']['Provincia'].unique()

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Córdoba', 'Corrientes', 'Entre Ríos', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén',
       'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucumán'],
      dtype=object)

In [14]:
# Se procede a reemplazar valores cuya diferencia es un acento
dataframes['Penetracion-hogares']['Provincia'] = dataframes['Penetracion-hogares']['Provincia'].replace('Córdoba', 'Cordoba')
dataframes['Penetracion-hogares']['Provincia'] = dataframes['Penetracion-hogares']['Provincia'].replace('Entre Ríos', 'Entre Rios')
dataframes['Penetracion-hogares']['Provincia'] = dataframes['Penetracion-hogares']['Provincia'].replace('Neuquén', 'Neuquen')
dataframes['Penetracion-hogares']['Provincia'] = dataframes['Penetracion-hogares']['Provincia'].replace('Río Negro', 'Rio Negro')
dataframes['Penetracion-hogares']['Provincia'] = dataframes['Penetracion-hogares']['Provincia'].replace('Tucumán', 'Tucuman')
dataframes['Penetracion-hogares']['Provincia'].unique()

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Cordoba', 'Corrientes', 'Entre Rios', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquen',
       'Rio Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucuman'],
      dtype=object)

In [ ]:
# Procedemos a ordenar las filas en orden ascendente por las columnas [Año y Trimestre]
dataframes['Penetracion-hogares'] = dataframes['Penetracion-hogares'].sort_values(by=['Año', 'Trimestre']).reset_index(drop=True)

In [16]:
dataframes['Penetracion-hogares'].head()

,Año,Trimestre,Provincia,Accesos por cada 100 hogares
0,2014,1,Buenos Aires,54.570073
1,2014,1,Capital Federal,112.254123
2,2014,1,Catamarca,24.251751
3,2014,1,Chaco,20.718539
4,2014,1,Chubut,41.235218


In [5]:
dataframes['Accesos_tecnologia_localidad'].head()

,Provincia,Partido,Localidad,Tecnologia,Link Indec,Accesos
0,BUENOS AIRES,25 de Mayo,25 de Mayo,ADSL,6854100,755.0
1,BUENOS AIRES,25 de Mayo,25 de Mayo,CABLEMODEM,6854100,4600.0
2,BUENOS AIRES,25 de Mayo,25 de Mayo,FIBRA OPTICA,6854100,2.0
3,BUENOS AIRES,25 de Mayo,25 de Mayo,SATELITAL,6854100,742.0
4,BUENOS AIRES,25 de Mayo,25 de Mayo,WIRELESS,6854100,727.0


In [6]:
# Se procede a revisar los valores nulos por columna, filas duplicadas y tipo de dato
tercer_dataframe = list(dataframes.keys())[2]
print('3er dataframe: ', tercer_dataframe)
print('-' * 50)
print('Cantidad de nulos por columna: \n', dataframes['Accesos_tecnologia_localidad'].isnull().sum())
print('-' * 50)
print('Cantidad de filas duplicadas: ', dataframes['Accesos_tecnologia_localidad'].duplicated().sum())
print('-' * 50)
print('Tipos de datos: \n', dataframes['Accesos_tecnologia_localidad'].dtypes)

3er dataframe:  Accesos_tecnologia_localidad
--------------------------------------------------
Cantidad de nulos por columna: 
 Provincia     0
Partido       0
Localidad     0
Tecnologia    0
Link Indec    0
Accesos       6
dtype: int64
--------------------------------------------------
Cantidad de filas duplicadas:  0
--------------------------------------------------
Tipos de datos: 
 Provincia      object
Partido        object
Localidad      object
Tecnologia     object
Link Indec     object
Accesos       float64
dtype: object


In [7]:
dataframes['Accesos_tecnologia_localidad']['Provincia'].unique()

array(['BUENOS AIRES', 'CABA', 'CATAMARCA', 'CHACO', 'CHUBUT', 'CORDOBA',
       'CORRIENTES', 'ENTRE RIOS', 'FORMOSA', 'JUJUY', 'LA PAMPA',
       'LA RIOJA', 'MENDOZA', 'MISIONES', 'NEUQUEN', 'RIO NEGRO', 'SALTA',
       'SAN JUAN', 'SAN LUIS', 'SANTA CRUZ', 'SANTA FE',
       'SANTIAGO DEL ESTERO', 'TIERRA DEL FUEGO', 'TUCUMAN'], dtype=object)

In [8]:
# Se normalizan los valores.
dataframes['Accesos_tecnologia_localidad']['Provincia'] = dataframes['Accesos_tecnologia_localidad']['Provincia'].str.title()
dataframes['Accesos_tecnologia_localidad']['Provincia'].unique()

array(['Buenos Aires', 'Caba', 'Catamarca', 'Chaco', 'Chubut', 'Cordoba',
       'Corrientes', 'Entre Rios', 'Formosa', 'Jujuy', 'La Pampa',
       'La Rioja', 'Mendoza', 'Misiones', 'Neuquen', 'Rio Negro', 'Salta',
       'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe',
       'Santiago Del Estero', 'Tierra Del Fuego', 'Tucuman'], dtype=object)

In [15]:
# Reemplazamos el valor de 'Caba' por 'Capital Federal'
dataframes['Accesos_tecnologia_localidad']['Provincia'] = dataframes['Accesos_tecnologia_localidad']['Provincia'].replace('Caba', 'Capital Federal')
dataframes['Accesos_tecnologia_localidad']['Provincia'].unique()

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Cordoba', 'Corrientes', 'Entre Rios', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquen',
       'Rio Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucuman'],
      dtype=object)

In [9]:
# Se procedio a identificar cuales eran las filas con valores nulos
fila, columna = np.where(dataframes['Accesos_tecnologia_localidad'].isna())
dataframes['Accesos_tecnologia_localidad'].iloc[fila]

,Provincia,Partido,Localidad,Tecnologia,Link Indec,Accesos
3755,Corrientes,Ituzaingó,Colonia Liebig's,Otros,18084010,NaN
4101,Entre Rios,Tala,Rosario del Tala,Otros,30091100,NaN
5744,Salta,Cafayate,Tolombón,Otros,66021020,NaN
6005,Salta,San Carlos,Animaná,Otros,66154020,NaN
6010,Salta,San Carlos,San Carlos,Otros,66154040,NaN
7372,Santiago Del Estero,Choya,Frías,Otros,86063040,NaN


In [10]:
moda_accesos = dataframes['Accesos_tecnologia_localidad']['Accesos'].mode()
print('La moda para la columna "accesos" es: ', moda_accesos[0])

La moda para la columna "accesos" es:  1.0


In [11]:
# Tratando de una variable numerica discreta se procedio a rellenar los valores nulos con la moda
dataframes['Accesos_tecnologia_localidad']['Accesos'].fillna(moda_accesos[0], inplace=True)
dataframes['Accesos_tecnologia_localidad']['Accesos'].isna().value_counts()

Accesos
False    7753
Name: count, dtype: int64

In [12]:
dataframes['Accesos_tecnologia_localidad']['Tecnologia'].unique()

array(['ADSL', 'CABLEMODEM', 'FIBRA OPTICA', 'SATELITAL', 'WIRELESS',
       'Otros', 'DIAL UP', 'OTROS', 'WIMAX', 'Cablemodem'], dtype=object)

In [13]:
dataframes['Accesos_tecnologia_localidad']['Tecnologia'] = dataframes['Accesos_tecnologia_localidad']['Tecnologia'].replace('Otros', 'OTROS')
dataframes['Accesos_tecnologia_localidad']['Tecnologia'] = dataframes['Accesos_tecnologia_localidad']['Tecnologia'].replace('Cablemodem', 'CABLEMODEM')
dataframes['Accesos_tecnologia_localidad']['Tecnologia'].unique()

array(['ADSL', 'CABLEMODEM', 'FIBRA OPTICA', 'SATELITAL', 'WIRELESS',
       'OTROS', 'DIAL UP', 'WIMAX'], dtype=object)

In [14]:
dataframes['Accesos Por Tecnología'].head()

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2024,2,Buenos Aires,214055.0,2722466.0,1849476.0,138638.0,64745.0,4989380.0
1,2024,2,Capital Federal,54102.0,1144781.0,230402.0,4493.0,29821.0,1463599.0
2,2024,2,Catamarca,4951.0,10303.0,58355.0,1384.0,81.0,75074.0
3,2024,2,Chaco,9448.0,57935.0,68944.0,8407.0,2358.0,147092.0
4,2024,2,Chubut,25955.0,80704.0,26516.0,31118.0,9930.0,174223.0


In [27]:
# Se procede a revisar los valores nulos por columna, filas duplicadas y tipo de dato
septimo_dataframe = list(dataframes.keys())[6]
print('6to dataframe: ', septimo_dataframe)
print('-' * 50)
print('Cantidad de nulos por columna: \n', dataframes['Accesos Por Tecnología'].isnull().sum())
print('-' * 50)
print('Cantidad de filas duplicadas: ', dataframes['Accesos Por Tecnología'].duplicated().sum())
print('-' * 50)
print('Tipos de datos: \n', dataframes['Accesos Por Tecnología'].dtypes)

6to dataframe:  Accesos Por Tecnología
--------------------------------------------------
Cantidad de nulos por columna: 
 Año             1
Trimestre       1
Provincia       2
ADSL            2
Cablemodem      2
Fibra óptica    2
Wireless        2
Otros           2
Total           2
dtype: int64
--------------------------------------------------
Cantidad de filas duplicadas:  0
--------------------------------------------------
Tipos de datos: 
 Año              object
Trimestre        object
Provincia        object
ADSL            float64
Cablemodem      float64
Fibra óptica    float64
Wireless        float64
Otros           float64
Total           float64
dtype: object


In [28]:
# Se procedio a identificar cuales eran las filas con valores nulos
fila, columna = np.where(dataframes['Accesos Por Tecnología'].isna())
dataframes['Accesos Por Tecnología'].iloc[fila]

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008,*,Los datos provinciales no coinciden a nivel na...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Se proceden a eliminar los valores nulos
dataframes['Accesos Por Tecnología'].dropna(inplace= True)

In [30]:
# Se procede a revisar los valores nulos por columna, filas duplicadas y tipo de dato
sexto_dataframe = list(dataframes.keys())[6]
print('6to dataframe: ', sexto_dataframe)
print('-' * 50)
print('Cantidad de nulos por columna: \n', dataframes['Accesos Por Tecnología'].isnull().sum())
print('-' * 50)
print('Cantidad de filas duplicadas: ', dataframes['Accesos Por Tecnología'].duplicated().sum())
print('-' * 50)
print('Tipos de datos: \n', dataframes['Accesos Por Tecnología'].dtypes)

6to dataframe:  Accesos Por Tecnología
--------------------------------------------------
Cantidad de nulos por columna: 
 Año             0
Trimestre       0
Provincia       0
ADSL            0
Cablemodem      0
Fibra óptica    0
Wireless        0
Otros           0
Total           0
dtype: int64
--------------------------------------------------
Cantidad de filas duplicadas:  0
--------------------------------------------------
Tipos de datos: 
 Año              object
Trimestre        object
Provincia        object
ADSL            float64
Cablemodem      float64
Fibra óptica    float64
Wireless        float64
Otros           float64
Total           float64
dtype: object


In [31]:
# Procedemos a explorar los valores unicos de la columna 'trimestre'
dataframes['Accesos Por Tecnología']['Trimestre'].unique()

array([2, 1, 4, 3, '3 *', '2 *', '1 *'], dtype=object)

In [32]:
# Se procede a reemplazar valores 
dataframes['Accesos Por Tecnología']['Trimestre'] = dataframes['Accesos Por Tecnología']['Trimestre'].replace('3 *', '3')
dataframes['Accesos Por Tecnología']['Trimestre'] = dataframes['Accesos Por Tecnología']['Trimestre'].replace('2 *', '2')
dataframes['Accesos Por Tecnología']['Trimestre'] = dataframes['Accesos Por Tecnología']['Trimestre'].replace('1 *', '1')
dataframes['Accesos Por Tecnología']['Trimestre'].unique()

array([2, 1, 4, 3, '3', '2', '1'], dtype=object)

In [33]:
# Se convirtieron los valores a formato a numerico con el obj de unificar
dataframes['Accesos Por Tecnología']['Trimestre'] = pd.to_numeric(dataframes['Accesos Por Tecnología']['Trimestre'], errors= 'coerce')
dataframes['Accesos Por Tecnología']['Trimestre'].unique()

array([2, 1, 4, 3])

In [34]:
# Procedemos a explorar los valores unicos de la columna 'Año'
dataframes['Accesos Por Tecnología']['Año'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, '2019 *', 2018, 2017, 2016,
       2015, 2014], dtype=object)

In [35]:
# Se procede a reemplazar valores 
dataframes['Accesos Por Tecnología']['Año'] = dataframes['Accesos Por Tecnología']['Año'].replace('2019 *', '2019')
dataframes['Accesos Por Tecnología']['Año'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, '2019', 2018, 2017, 2016, 2015,
       2014], dtype=object)

In [36]:
# Se convirtieron los valores a formato a numerico con el obj de unificar
dataframes['Accesos Por Tecnología']['Año'] = pd.to_numeric(dataframes['Accesos Por Tecnología']['Año'], errors= 'coerce')
dataframes['Accesos Por Tecnología']['Año'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014])

In [37]:
# Se imprimen los tipos de datos por columna
dataframes['Accesos Por Tecnología'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1007 entries, 0 to 1006
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Año           1007 non-null   int64  
 1   Trimestre     1007 non-null   int64  
 2   Provincia     1007 non-null   object 
 3   ADSL          1007 non-null   float64
 4   Cablemodem    1007 non-null   float64
 5   Fibra óptica  1007 non-null   float64
 6   Wireless      1007 non-null   float64
 7   Otros         1007 non-null   float64
 8   Total         1007 non-null   float64
dtypes: float64(6), int64(2), object(1)
memory usage: 78.7+ KB


In [38]:
dataframes['Accesos Por Tecnología']['Provincia'].unique()

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Córdoba', 'Corrientes', 'Entre Ríos', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén',
       'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucumán'],
      dtype=object)

In [39]:
# Se procede a reemplazar valores cuya diferencia es un acento
dataframes['Accesos Por Tecnología']['Provincia'] = dataframes['Accesos Por Tecnología']['Provincia'].replace('Córdoba', 'Cordoba')
dataframes['Accesos Por Tecnología']['Provincia'] = dataframes['Accesos Por Tecnología']['Provincia'].replace('Entre Ríos', 'Entre Rios')
dataframes['Accesos Por Tecnología']['Provincia'] = dataframes['Accesos Por Tecnología']['Provincia'].replace('Neuquén', 'Neuquen')
dataframes['Accesos Por Tecnología']['Provincia'] = dataframes['Accesos Por Tecnología']['Provincia'].replace('Río Negro', 'Rio Negro')
dataframes['Accesos Por Tecnología']['Provincia'] = dataframes['Accesos Por Tecnología']['Provincia'].replace('Tucumán', 'Tucuman')
dataframes['Accesos Por Tecnología']['Provincia'].unique()

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Cordoba', 'Corrientes', 'Entre Rios', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquen',
       'Rio Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucuman'],
      dtype=object)

In [40]:
# Procedemos a ordenar las filas en orden ascendente por las columnas [Año y Trimestre]
dataframes['Accesos Por Tecnología'] = dataframes['Accesos Por Tecnología'].sort_values(by=['Año', 'Trimestre']).reset_index(drop=True)

In [41]:
dataframes['Accesos Por Tecnología'].head()

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2014,1,Buenos Aires,1567685.0,1000879.0,120960.0,16528.0,33824.0,2739876.0
1,2014,1,Capital Federal,445569.0,820161.0,17767.0,1866.0,14555.0,1299918.0
2,2014,1,Catamarca,24126.0,46.0,17.0,0.0,43.0,24232.0
3,2014,1,Chaco,52971.0,9497.0,30.0,0.0,58.0,62556.0
4,2014,1,Chubut,51066.0,539.0,866.0,10776.0,6974.0,70221.0


### ETL para KPI 2 (Velocidad_sin_Rangos, Velocidad % por prov, Totales VMD)

#### Velocidad_sin_Rangos

In [42]:
# Imprimir cada dataframe
for dataframe in dataframes.keys():
    print(dataframe)

Acc_vel_loc_sinrangos
Velocidad_sin_Rangos
Accesos_tecnologia_localidad
Velocidad % por prov
Totales VMD
Totales Accesos Por Tecnología
Accesos Por Tecnología
Penetración-poblacion
Penetracion-hogares
Penetracion-totales
Totales Accesos por rango
Accesos por rangos
Dial-BAf
Totales Dial-BAf
Ingresos 


In [43]:
# Previsualizacion del dataframe
dataframes['Velocidad_sin_Rangos'].head()

,Año,Trimestre,Provincia,Velocidad,Accesos
0,2024,2,BUENOS AIRES,75.0,1062
1,2024,2,BUENOS AIRES,59.0,59
2,2024,2,BUENOS AIRES,480.0,5
3,2024,2,BUENOS AIRES,3.5,41735
4,2024,2,BUENOS AIRES,18.0,1042


In [44]:
# Se procede a renombrar las columnas correctamente
dataframes['Velocidad_sin_Rangos'].rename(columns= {'Velocidad': 'Velocidad (Mbps)'}, inplace=True)
dataframes['Velocidad_sin_Rangos'].head()

,Año,Trimestre,Provincia,Velocidad (Mbps),Accesos
0,2024,2,BUENOS AIRES,75.0,1062
1,2024,2,BUENOS AIRES,59.0,59
2,2024,2,BUENOS AIRES,480.0,5
3,2024,2,BUENOS AIRES,3.5,41735
4,2024,2,BUENOS AIRES,18.0,1042


In [45]:
# Se procede a revisar los valores nulos por columna, filas duplicadas y tipo de dato
segundo_dataframe = list(dataframes.keys())[1]
print('2do dataframe: ', segundo_dataframe)
print('-' * 50)
print('Cantidad de nulos por columna: \n', dataframes['Velocidad_sin_Rangos'].isnull().sum())
print('-' * 50)
print('Cantidad de filas duplicadas: ', dataframes['Velocidad_sin_Rangos'].duplicated().sum())
print('-' * 50)
print('Tipos de datos: \n', dataframes['Velocidad_sin_Rangos'].dtypes)

2do dataframe:  Velocidad_sin_Rangos
--------------------------------------------------
Cantidad de nulos por columna: 
 Año                  0
Trimestre            0
Provincia            0
Velocidad (Mbps)    10
Accesos              0
dtype: int64
--------------------------------------------------
Cantidad de filas duplicadas:  0
--------------------------------------------------
Tipos de datos: 
 Año                   int64
Trimestre             int64
Provincia            object
Velocidad (Mbps)    float64
Accesos               int64
dtype: object


In [46]:
# Se revisan los valores unicos por columna
dataframes['Velocidad_sin_Rangos']['Provincia'].unique()

array(['BUENOS AIRES', 'CABA', 'CATAMARCA', 'CHACO', 'CHUBUT', 'CORDOBA',
       'CORRIENTES', 'ENTRE RIOS', 'FORMOSA', 'JUJUY', 'LA PAMPA',
       'LA RIOJA', 'MENDOZA', 'MISIONES', 'NEUQUEN', 'RIO NEGRO', 'SALTA',
       'SAN JUAN', 'SAN LUIS', 'SANTA CRUZ', 'SANTA FE',
       'SANTIAGO DEL ESTERO', 'TIERRA DEL FUEGO', 'TUCUMAN',
       'Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Córdoba', 'Corrientes', 'Entre Ríos', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén',
       'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucumán'],
      dtype=object)

In [47]:
# Se normalizan los valores.
dataframes['Velocidad_sin_Rangos']['Provincia'] = dataframes['Velocidad_sin_Rangos']['Provincia'].str.title()
dataframes['Velocidad_sin_Rangos']['Provincia'].unique()

array(['Buenos Aires', 'Caba', 'Catamarca', 'Chaco', 'Chubut', 'Cordoba',
       'Corrientes', 'Entre Rios', 'Formosa', 'Jujuy', 'La Pampa',
       'La Rioja', 'Mendoza', 'Misiones', 'Neuquen', 'Rio Negro', 'Salta',
       'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe',
       'Santiago Del Estero', 'Tierra Del Fuego', 'Tucuman',
       'Capital Federal', 'Córdoba', 'Entre Ríos', 'Neuquén', 'Río Negro',
       'Tucumán'], dtype=object)

In [48]:
# Se procede a reemplazar valores cuya diferencia es un acento
dataframes['Velocidad_sin_Rangos']['Provincia'] = dataframes['Velocidad_sin_Rangos']['Provincia'].replace('Córdoba', 'Cordoba')
dataframes['Velocidad_sin_Rangos']['Provincia'] = dataframes['Velocidad_sin_Rangos']['Provincia'].replace('Entre Ríos', 'Entre Rios')
dataframes['Velocidad_sin_Rangos']['Provincia'] = dataframes['Velocidad_sin_Rangos']['Provincia'].replace('Neuquén', 'Neuquen')
dataframes['Velocidad_sin_Rangos']['Provincia'] = dataframes['Velocidad_sin_Rangos']['Provincia'].replace('Río Negro', 'Rio Negro')
dataframes['Velocidad_sin_Rangos']['Provincia'] = dataframes['Velocidad_sin_Rangos']['Provincia'].replace('Tucumán', 'Tucuman')
dataframes['Velocidad_sin_Rangos']['Provincia'].unique()

array(['Buenos Aires', 'Caba', 'Catamarca', 'Chaco', 'Chubut', 'Cordoba',
       'Corrientes', 'Entre Rios', 'Formosa', 'Jujuy', 'La Pampa',
       'La Rioja', 'Mendoza', 'Misiones', 'Neuquen', 'Rio Negro', 'Salta',
       'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe',
       'Santiago Del Estero', 'Tierra Del Fuego', 'Tucuman',
       'Capital Federal'], dtype=object)

In [49]:
# Se procedio a identificar cuales eran las filas con valores nulos
fila, columna = np.where(dataframes['Velocidad_sin_Rangos'].isna())
dataframes['Velocidad_sin_Rangos'].iloc[fila]

,Año,Trimestre,Provincia,Velocidad (Mbps),Accesos
1155,2024,1,Cordoba,NaN,1
1644,2024,1,Santa Fe,NaN,9
3753,2023,2,Cordoba,NaN,1
4239,2023,2,Santa Fe,NaN,9
6303,2022,3,Cordoba,NaN,1
6785,2022,3,Santa Fe,NaN,9
9255,2021,3,Buenos Aires,NaN,4
10031,2021,2,Buenos Aires,NaN,4
10794,2021,1,Buenos Aires,NaN,4
11538,2020,4,Buenos Aires,NaN,4


In [50]:
# Se almaceno en una variable el valor del promedio de la columna
mean_velocidad = dataframes['Velocidad_sin_Rangos']['Velocidad (Mbps)'].mean()
print('El valor promedio en la columna "Velocidad (Mbps)" es: ', mean_velocidad)

El valor promedio en la columna "Velocidad (Mbps)" es:  96.17115566387623


In [51]:
# Se rellenaron los valores nulos con la variable de valor promedio
dataframes['Velocidad_sin_Rangos']['Velocidad (Mbps)'].fillna(mean_velocidad, inplace=True)
dataframes['Velocidad_sin_Rangos']['Velocidad (Mbps)'].isna().value_counts()

Velocidad (Mbps)
False    18884
Name: count, dtype: int64

In [52]:
# Procedemos a ordenar las filas en orden ascendente por las columnas [Año y Trimestre]
dataframes['Velocidad_sin_Rangos'] = dataframes['Velocidad_sin_Rangos'].sort_values(by=['Año', 'Trimestre']).reset_index(drop=True)

In [53]:
dataframes['Velocidad_sin_Rangos'].head()

,Año,Trimestre,Provincia,Velocidad (Mbps),Accesos
0,2017,4,Buenos Aires,0.256,114182
1,2017,4,Buenos Aires,0.512,3865
2,2017,4,Buenos Aires,1.000,35185
3,2017,4,Buenos Aires,1.200,19861
4,2017,4,Buenos Aires,2.200,29124


#### Velocidad % por prov

In [54]:
dataframes['Velocidad % por prov'].head()

,Año,Trimestre,Provincia,Mbps (Media de bajada)
0,2024,2,Buenos Aires,157.41
1,2024,2,Capital Federal,233.01
2,2024,2,Catamarca,97.38
3,2024,2,Chaco,107.76
4,2024,2,Chubut,21.67


In [55]:
# Se procede a revisar los valores nulos por columna, filas duplicadas y tipo de dato
cuarto_dataframe = list(dataframes.keys())[3]
print('4to dataframe: ', cuarto_dataframe)
print('-' * 50)
print('Cantidad de nulos por columna: \n', dataframes['Velocidad % por prov'].isnull().sum())
print('-' * 50)
print('Cantidad de filas duplicadas: ', dataframes['Velocidad % por prov'].duplicated().sum())
print('-' * 50)
print('Tipos de datos: \n', dataframes['Velocidad % por prov'].dtypes)

4to dataframe:  Velocidad % por prov
--------------------------------------------------
Cantidad de nulos por columna: 
 Año                       0
Trimestre                 0
Provincia                 0
Mbps (Media de bajada)    0
dtype: int64
--------------------------------------------------
Cantidad de filas duplicadas:  0
--------------------------------------------------
Tipos de datos: 
 Año                         int64
Trimestre                   int64
Provincia                  object
Mbps (Media de bajada)    float64
dtype: object


In [56]:
print(dataframes['Velocidad % por prov']['Año'].unique())
print(dataframes['Velocidad % por prov']['Provincia'].unique())
print(dataframes['Velocidad % por prov']['Trimestre'].unique())


[2024 2023 2022 2021 2020 2019 2018 2017 2016 2015 2014]
['Buenos Aires' 'Capital Federal' 'Catamarca' 'Chaco' 'Chubut' 'Córdoba'
 'Corrientes' 'Entre Ríos' 'Formosa' 'Jujuy' 'La Pampa' 'La Rioja'
 'Mendoza' 'Misiones' 'Neuquén' 'Río Negro' 'Salta' 'San Juan' 'San Luis'
 'Santa Cruz' 'Santa Fe' 'Santiago Del Estero' 'Tierra Del Fuego'
 'Tucumán']
[2 1 4 3]


In [57]:
# Se procede a reemplazar valores cuya diferencia es un acento
dataframes['Velocidad % por prov']['Provincia'] = dataframes['Velocidad % por prov']['Provincia'].replace('Córdoba', 'Cordoba')
dataframes['Velocidad % por prov']['Provincia'] = dataframes['Velocidad % por prov']['Provincia'].replace('Entre Ríos', 'Entre Rios')
dataframes['Velocidad % por prov']['Provincia'] = dataframes['Velocidad % por prov']['Provincia'].replace('Neuquén', 'Neuquen')
dataframes['Velocidad % por prov']['Provincia'] = dataframes['Velocidad % por prov']['Provincia'].replace('Río Negro', 'Rio Negro')
dataframes['Velocidad % por prov']['Provincia'] = dataframes['Velocidad % por prov']['Provincia'].replace('Tucumán', 'Tucuman')
dataframes['Velocidad % por prov']['Provincia'].unique()

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Cordoba', 'Corrientes', 'Entre Rios', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquen',
       'Rio Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucuman'],
      dtype=object)

In [58]:
# Procedemos a ordenar las filas en orden ascendente por las columnas [Año y Trimestre]
dataframes['Velocidad % por prov'] = dataframes['Velocidad % por prov'].sort_values(by=['Año', 'Trimestre']).reset_index(drop=True)

In [59]:
dataframes['Velocidad % por prov'].head()

,Año,Trimestre,Provincia,Mbps (Media de bajada)
0,2014,1,Buenos Aires,3.733133
1,2014,1,Capital Federal,4.024082
2,2014,1,Catamarca,2.802734
3,2014,1,Chaco,3.255837
4,2014,1,Chubut,3.175917


#### Totales VMD

In [60]:
dataframes['Totales VMD'].head()

,Año,Trimestre,Mbps (Media de bajada),Trimestre.1
0,2024,2,139.25,Abr-Jun 2024
1,2024,1,139.15,Ene-Mar 2024
2,2023,4,139.04,Oct-Dic 2023
3,2023,3,129.67,Jul-Sept 2023
4,2023,2,123.95,Abr-Jun 2023


In [61]:
# Se procede a revisar los valores nulos por columna, filas duplicadas y tipo de dato
quinto_dataframe = list(dataframes.keys())[4]
print('5to dataframe: ', quinto_dataframe)
print('-' * 50)
print('Cantidad de nulos por columna: \n', dataframes['Totales VMD'].isnull().sum())
print('-' * 50)
print('Cantidad de filas duplicadas: ', dataframes['Totales VMD'].duplicated().sum())
print('-' * 50)
print('Tipos de datos: \n', dataframes['Totales VMD'].dtypes)

5to dataframe:  Totales VMD
--------------------------------------------------
Cantidad de nulos por columna: 
 Año                       0
Trimestre                 0
Mbps (Media de bajada)    0
Trimestre.1               0
dtype: int64
--------------------------------------------------
Cantidad de filas duplicadas:  0
--------------------------------------------------
Tipos de datos: 
 Año                         int64
Trimestre                   int64
Mbps (Media de bajada)    float64
Trimestre.1                object
dtype: object


In [62]:
# Se elimina la columna 'Trimestre.1' ya que no nos aporta mas informacion de la ya propuesta
dataframes['Totales VMD'].drop(columns='Trimestre.1', inplace=True)
dataframes['Totales VMD'].head()

,Año,Trimestre,Mbps (Media de bajada)
0,2024,2,139.25
1,2024,1,139.15
2,2023,4,139.04
3,2023,3,129.67
4,2023,2,123.95


In [63]:
# Procedemos a ordenar las filas en orden ascendente por las columnas [Año y Trimestre]
dataframes['Totales VMD'] = dataframes['Totales VMD'].sort_values(by=['Año', 'Trimestre']).reset_index(drop=True)

In [64]:
dataframes['Totales VMD'].head()

,Año,Trimestre,Mbps (Media de bajada)
0,2014,1,3.617127
1,2014,2,3.758712
2,2014,3,3.867265
3,2014,4,4.156888
4,2015,1,4.350279


### ETL para KPI 3 (Accesos_tecnologia_localidad, Totales Accesos Por Tecnología, Accesos por tecnología.)

#### Totales Accesos Por Tecnología

In [65]:
dataframes['Totales Accesos Por Tecnología'].head()

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo
0,2024,2,733491,5867504,4169958,593197,191957,11556107,Abr-Jun 2024
1,2024,1,774475,5986957,4015101,598682,257941,11633156,Ene-Mar 2024
2,2023,4,836390,6022532,3908183,585760,194796,11547661,Oct-Dic 2023
3,2023,3,897895,6018832,3708718,581436,200027,11406908,Jul-Sept 2023
4,2023,2,1006509,5997149,3463988,581823,202428,11251897,Abr-Jun 2023


In [66]:
# Se procede a revisar los valores nulos por columna, filas duplicadas y tipo de dato
sexto_dataframe = list(dataframes.keys())[5]
print('5to dataframe: ', sexto_dataframe)
print('-' * 50)
print('Cantidad de nulos por columna: \n', dataframes['Totales Accesos Por Tecnología'].isnull().sum())
print('-' * 50)
print('Cantidad de filas duplicadas: ', dataframes['Totales Accesos Por Tecnología'].duplicated().sum())
print('-' * 50)
print('Tipos de datos: \n', dataframes['Totales Accesos Por Tecnología'].dtypes)

5to dataframe:  Totales Accesos Por Tecnología
--------------------------------------------------
Cantidad de nulos por columna: 
 Año             0
Trimestre       0
ADSL            0
Cablemodem      0
Fibra óptica    0
Wireless        0
Otros           0
Total           0
Periodo         0
dtype: int64
--------------------------------------------------
Cantidad de filas duplicadas:  0
--------------------------------------------------
Tipos de datos: 
 Año              int64
Trimestre        int64
ADSL             int64
Cablemodem       int64
Fibra óptica     int64
Wireless         int64
Otros            int64
Total            int64
Periodo         object
dtype: object


In [67]:
# Procedemos a ordenar las filas en orden ascendente por las columnas [Año y Trimestre]
dataframes['Totales Accesos Por Tecnología'] = dataframes['Totales Accesos Por Tecnología'].sort_values(by=['Año', 'Trimestre']).reset_index(drop=True)

In [68]:
dataframes['Totales Accesos Por Tecnología'].head()

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo
0,2014,1,3697066,2407330,150323,70749,72930,6398398,Ene-Mar 2014
1,2014,2,3708882,2461670,149363,72405,72148,6464468,Abr-Jun 2014
2,2014,3,3714764,2569868,155494,85096,70049,6595271,Jul-Sept 2014
3,2014,4,3764038,2536219,149682,76984,71573,6598496,Oct-Dic 2014
4,2015,1,3756153,2668248,168188,79098,66045,6737732,Ene-Mar 2015


#### Ingresos

In [69]:
dataframes['Ingresos '].head()

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2024,2,4.420322e+08,Abr-Jun 2024
1,2024,1,3.461990e+08,Ene-Mar 2024
2,2023,4,1.673760e+08,Oct-Dic 2023
3,2023,3,1.331066e+08,Jul-Sept 2023
4,2023,2,1.180603e+08,Jun-Mar 2023


In [70]:
# Se procede a revisar los valores nulos por columna, filas duplicadas y tipo de dato
ultimo_dataframe = list(dataframes.keys())[14]
print('5to dataframe: ', ultimo_dataframe)
print('-' * 50)
print('Cantidad de nulos por columna: \n', dataframes['Ingresos '].isnull().sum())
print('-' * 50)
print('Cantidad de filas duplicadas: ', dataframes['Ingresos '].duplicated().sum())
print('-' * 50)
print('Tipos de datos: \n', dataframes['Ingresos '].dtypes)

5to dataframe:  Ingresos 
--------------------------------------------------
Cantidad de nulos por columna: 
 Año                          0
Trimestre                    0
Ingresos (miles de pesos)    0
Periodo                      0
dtype: int64
--------------------------------------------------
Cantidad de filas duplicadas:  0
--------------------------------------------------
Tipos de datos: 
 Año                            int64
Trimestre                      int64
Ingresos (miles de pesos)    float64
Periodo                       object
dtype: object


In [71]:
# Procedemos a ordenar las filas en orden ascendente por las columnas [Año y Trimestre]
dataframes['Ingresos '] = dataframes['Ingresos '].sort_values(by=['Año', 'Trimestre']).reset_index(drop=True)

In [73]:
dataframes['Ingresos '].head()

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2014,1,2.984054e+06,Ene-Mar 2014
1,2014,2,3.270816e+06,Abr-Jun 2014
2,2014,3,3.478638e+06,Jul-Sept 2014
3,2014,4,3.950441e+06,Oct-Dic 2014
4,2015,1,4.876385e+06,Ene-Mar 2015


### Exportaciones a formato 'csv'

#### Archivos del KPI 1

In [74]:
dataframes['Penetracion-hogares'].to_csv('./CSVs/kpi_1/penetracion_hogares.csv', index=False)

In [ ]:
dataframes['Accesos_tecnologia_localidad'].to_csv('./CSVs/kpi_1/accesos_tecnologia_localidad.csv', index=False)

In [76]:
dataframes['Accesos Por Tecnología'].to_csv('./CSVs/kpi_1/accesos_por_tecnologia.csv', index=False)

#### Archivos del KPI 2

In [77]:
dataframes['Velocidad_sin_Rangos'].to_csv('./CSVs/kpi_2/velocidad_sin_rangos.csv', index=False)

In [78]:
dataframes['Velocidad % por prov'].to_csv('./CSVs/kpi_2/velocidad_%_por_prov.csv', index=False)

In [79]:
dataframes['Totales VMD'].to_csv('./CSVs/kpi_2/totales_vmd.csv', index=False)

#### Archivos del KPI 3

In [80]:
dataframes['Accesos_tecnologia_localidad'].to_csv('./CSVs/kpi_3/accesos_tecnologia_localidad.csv', index=False)

In [81]:
dataframes['Accesos Por Tecnología'].to_csv('./CSVs/kpi_3/accesos_por_tecnologia.csv', index=False)

In [82]:
dataframes['Totales Accesos Por Tecnología'].to_csv('./CSVs/kpi_3/totales_accesos_por_tecnologia.csv', index=False)

In [83]:
dataframes['Ingresos '].to_csv('./CSVs/kpi_3/ingresos.csv', index=False)